from calibrate import CameraCalibrator
import glob
import cv2
import numpy as np

calibration_images = glob.glob('./camera_cal/calibration*.jpg')
calibrator = CameraCalibrator(calibration_images,
                                      9, 6, use_existing_camera_coefficients=True)
print (calibrator._calibrate())

length, width = 720, 1280

# Define shape for perspective transformation
lb = (130,width)
rb = (1235,width)
lt = (560,465)
rt = (730,465)

offset = 200
src = np.float32([lb, lt, rt, rb])
d_lb = (offset,width)
d_rb = (offset, 0)
d_lt = (length - offset, 0)
d_rt = (length - offset, width)
dst = np.float32([d_lb, d_rb, d_lt, d_rt])

# get M, the transform matrix
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst,src)
# Perspective transform

print(np.vectorize("%.3f".__mod__)(M))

In [ ]:
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

In [ ]:
import pipeline as p
import patches

img = mpimg.imread("./test_images/vid1/223.jpg")

dst = p.undistort(img)
    
ool, oor = 46,58
warped, sure = p.get_birdView(dst) #np.divide(.astype(float), 255.0)
edges = p.get_lines(warped)

sm = np.array(edges[-edges.shape[0] // 4:, :])
wm = np.array(warped[-warped.shape[0] // 4:, :])

ploty = np.linspace(0, sm.shape[0]-1, num=sm.shape[0])

#plt.imshow(np.multiply(sm, 5))
#plt.show()

de2 = np.dstack((sm,sm, sm)).astype(uint8)
left_fitx, right_fitx, ff, ool, oor = patches.find_curves(sm, ploty, ool, oor, de2=de2, verbose=5)

plt.imshow(ff)
plt.show()

plt.imshow(de2)
plt.show()

print (ool-oor)
    
if (ool is not None) and (oor is not None) and ool-oor>15:
    print (reset)
    ool, oor = None, None

if (ool is not None) and (oor is not None) and ((oor-ool>16) or (oor-ool<5)):
    print ("reset")
    ool, oor = None, None

layer = p.fill_lane_lines(np.dstack((edges,edges, edges)).astype(uint8), np.add(ploty, 300), left_fitx, right_fitx)

plt.imshow(layer)
plt.show()

In [ ]:
#disable 
import pipeline as p
import glob
import patches

f=0

show_bearing_point = True
ploty = None
ool, oor = None, None

#for img in map( mpimg.imread, p.hard): #
for img in map( mpimg.imread, sorted(glob.glob('./test_images/vid1/*.jpg'))):
    f=f+1
    print (f)
        
    dst = p.undistort(img)
    
    warped, sure = p.get_birdView(dst) #np.divide(.astype(float), 255.0)
    edges = p.get_lines(warped)
    
    #mpimg.imsave("/home/veon/edu/udacity/CarND-Advanced-Lane-Lines/test_images/edges"+str(f)+".png", edges, cmap="gray")
    
    sm = np.array(edges[-edges.shape[0] // 4:, :])
    wm = np.array(warped[-warped.shape[0] // 4:, :])
    
    if ploty is None:
        ploty = np.linspace(0, sm.shape[0]-1, num=sm.shape[0])
    
    #plt.imshow(np.multiply(sm, 5))
    #plt.show()

    de2 = np.dstack((sm,sm, sm)).astype(uint8)
#    try:
    left_fitx, right_fitx, ff, ool, oor = patches.find_curves(sm, ploty, ool, oor, de2=de2, verbose=3)
    
    #plt.imshow(ff), plt.show()
    
    #plt.imshow(de2), plt.show()
    
    if (ool is not None) and (oor is not None) and ((oor-ool>16) or (oor-ool<5)):
        print ("reset")
        ool, oor = None, None

    if left_fitx is None or right_fitx is None:
        continue
        
    layer = p.fill_lane_lines(np.dstack((edges,edges, edges)).astype(uint8), np.add(ploty, 300), left_fitx, right_fitx)

#    except:
#        plt.imshow(sm)
#        plt.show()
    
    #plt.imshow(out_img)
    #plt.show()
    if f > 222:
        p.draw_marks(dst, layer)